https://towardsdatascience.com/machine-learning-on-multioutput-datasets-a-quick-guide-ebeba81b97d1  
https://www.kaggle.com/code/vijaymukkala/multi-output-multi-classification-problem  
https://medium.com/@cactuscode/multioutput-multiclass-classification-b0737a0693ec  
https://www.geeksforgeeks.org/multiclass-vs-multioutput-algorithms-in-machine-learning/